# Explorando las formaciones frente a errores en la percepción
## A partir de "Maneuvering and robustness issues in undirecter displacemente-consensus-based formation control" de Héctor García de Marina
En el artículo se modelan los errores en la percepción como un error de escalado y una matriz de rotación que representan un alineado incorrecto en la medida de las posiciones.
$$ (p_i-p_j)_{measurement}=aR(p_i-p_j)$$
$$ a \in \mathcal{R}, R \in \mathcal{SO}(m)$$

### Carga de módulos necesarios

In [2]:
import RobotModels as rob
import numpy as np
import matplotlib.pyplot as plt
import graph_utils as graf
import time
from matplotlib import animation
from matplotlib.animation import FuncAnimation
from matplotlib.animation import PillowWriter
import robot_plot_utils as rpu

## Formación inicial con 2 agentes
Generación de las posiciones iniciales y el grafo.

In [34]:
#Posiciones iniciales
pi=np.array([[-10,0],[10,0]])
p=np.zeros([2,2])
numnodos,m=np.shape(pi)
#Formacion
#zstar=np.array([-5,-5])
#Grafo
Z=np.array([[1,2]])
numarcos,m=np.shape(Z)
n,m= np.shape(pi)
for i in range(n):
    plt.plot(pi[i,0],pi[i,1],'o')

rpu.dibuja_grafo_2D(pi,Z)

Im=np.eye(2)
pesos=np.ones([numnodos,numnodos])
L=graf.matriz_laplaciana(Z,pesos,numnodos,numarcos)

Lbar=np.kron(L,Im)
print("Lbar: ")
print(Lbar)

b=np.zeros([numnodos,numnodos])
for k in range(numarcos):
    i=Z[k,0]
    j=Z[k,1]
    b[i-1,j-1]=np.random.rand(1,1)
    b[j-1,i-1]=np.random.rand(1,1)
    #b[i-1,j-1]=1
print(b)

p=np.zeros([2*n,1])
#print(np.shape(p))
pstar_s=graf.stack(pi)
pstar=pi
t=0
tf=100
dt=0.01
nptos=(int)((tf-t)/dt)
#print(nptos)
fig2 =plt.figure()
p=np.array([[-10,0],[10,0]])
posiciones=np.zeros([2*n , nptos])
k=0
#print( p)
pmeasured=np.zeros([2,2])
pmeasured[0,:]=p[1,:]-p[0,:]+b[0,1]*np.ones([1,2])
pmeasured[1,:]=p[0,:]-p[1,:]+b[1,0]*np.ones([1,2])
#print(pmeasured)

for k in range(nptos):
    pmeasured_s=graf.stack(pmeasured)
    #print("Pmeasured:")
    #print(pmeasured_s)   
    pdot=-np.dot(Lbar,pmeasured_s)
    #print("Pdot: ")
    #print(pdot)
    for i in range(0,numnodos,1):
        xdot,ydot,thetadot=rob.integrator_point_model(pdot[i],pdot[i+1])
        p[i,0]+=dt*xdot
        p[i,1]+=dt*ydot
    pmeasured[0,:]=p[1,:]-p[0,:]+b[0,1]*np.ones([1,2])
    #pmeasured[1,:]=p[0,:]-p[1,:]+b[1,0]*np.ones([1,2])
    pmeasured[1,:]=p[0,:]-p[1,:]
    p_s=graf.stack(p)
    posiciones[:,k]=p_s[:]
    t+=dt
print(posiciones[:,k])
colors_f=['bo','go','ro','co','yo','mo','ko','bo','go']
colors_i=['bx','gx','rx','cx','yx','mx','kx','bx','gx']
colors_l=['b.','g.','r','c','y','m','k','b:','g:']

j=0
for i in range(numnodos):
    plt.plot(posiciones[j,0],posiciones[j+1,0],colors_i[i])
    plt.plot(posiciones[j,nptos-1],posiciones[j+1,nptos-1],colors_f[i])
    plt.plot(posiciones[j,:],posiciones[j+1,:],colors_l[i])
    j=j+2
plt.grid()

Lbar: 
[[ 1.  0. -1. -0.]
 [ 0.  1. -0. -1.]
 [-1. -0.  1.  0.]
 [-0. -1.  0.  1.]]
[[0.         0.49135802]
 [0.64014747 0.        ]]


/tmp/ipykernel_51379/3871593167.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  b[i-1,j-1]=np.random.rand(1,1)
/tmp/ipykernel_51379/3871593167.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  b[j-1,i-1]=np.random.rand(1,1)


[-10.   0.   0.   0.]


In [8]:

#print(Lps)
#Revisar: Los 5 primeros salen bien pero el resto no
for k in range(nptos):
    pmeasured=np.array(
    pdot=-np.dot(Lbar,pmeasured)
    for i in range(0,2*n,2):
        xdot,ydot,thetadot=rob.integrator_point_model(pdot[i],pdot[i+1])
        p[i]+=dt*xdot
        p[i+1]+=dt*ydot
    posiciones[:,k]=p[:]
    t+=dt
print(posiciones[:,k])
colors_f=['bo','go','ro','co','yo','mo','ko','bo','go']
colors_i=['bx','gx','rx','cx','yx','mx','kx','bx','gx']
colors_l=['b','g','r','c','y','m','k','b:','g:']
Z=np.array([[1,2],[2,3],[3,4],[4,5],[5,6],[5,8],[8,7],[8,9]])
j=0
for i in range(n):
    plt.plot(posiciones[j,0],posiciones[j+1,0],colors_i[i])
    plt.plot(posiciones[j,nptos-1],posiciones[j+1,nptos-1],colors_f[i])
    plt.plot(posiciones[j,:],posiciones[j+1,:],colors_l[i])
    j=j+2
#rpu.dibuja_grafo_2D(pi,Z)
pfs=np.zeros([2*n,1])
pfs[:,0]=posiciones[:,nptos-1]
pinis=np.zeros([2*n,1])
pinis[:,0]=posiciones[:,0]
#print(pfs)
pf=graf.unstack(pfs,2)
pini=graf.unstack(pinis,2)
#print(pf)
rpu.dibuja_grafo_2D(pf,Z)
rpu.dibuja_grafo_2D(pini,Z)
plt.show()

SyntaxError: invalid syntax (2432617857.py, line 6)